In [1]:
#import libraries
import selenium
from selenium import webdriver
import pandas as pd

In [47]:
#Importing requests
import requests

# importing regex
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time


1) Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
#Getting user input
search_item= input('Enter the product that has to be searched : ')

Enter the product that has to be searched : guitar


In [4]:
search_item

'guitar'

In [5]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.amazon.in/')
time.sleep(1)

#enter search_item in the search field
driver.find_element(By.XPATH,'//input[@class="nav-input nav-progressive-attribute"]').send_keys(search_item)
time.sleep(2)

#click the search button
driver.find_element(By.XPATH,'//span[@class="nav-search-submit-text nav-sprite nav-progressive-attribute"]').click()
time.sleep(2)

2) In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
#collecting all the Product URLS here
urls = []
for i in range(0,2):
    product_urls=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")#collecting urls of all the products
    for i in product_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element(By.XPATH,"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [7]:
len(urls)

125

In [8]:
#initializing attributed need to be scraped
brand_name = []
product_name = []
price = []
return_exchange = []
expected_delivery = []
availability = []
product_url = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping data for brand name
    try:
        brand=driver.find_element(By.XPATH,"//div[@id='bylineInfo_feature_div']/div/a")
        brand_name.append(brand.text)
    except NoSuchElementException as e:
        brand_name.append("-")
    
    #Scraping data for product name
    try:
        prod=driver.find_element(By.XPATH,"//span[@id='productTitle']")
        product_name.append(prod.text)
    except NoSuchElementException as e:
        product_name.append("-")    
         
    #Scraping data for Price
    try:
        pri=driver.find_element(By.XPATH,"//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']/span")
        price.append(pri.text)
    except NoSuchElementException as e:
        price.append("-")
        
        
    #Scraping data for Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,"//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        return_exchange.append(ret.text)
    except NoSuchElementException as e:
        return_exchange.append("-")
        
     
    #Scraping data for Expected_Delivery
    try:
        delivery=driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']/b")
        expected_delivery.append(delivery.text)
    except NoSuchElementException as e:
        expected_delivery.append("-")
        
        
    #Scraping data for Availability
    try:
        avai=driver.find_element(By.XPATH,"//div[@id='availability']/span")
        availability.append(avai.text)
    except NoSuchElementException as e:
        availability.append("-")
        
    #product_url
    product_url.append(i)
    
#creating a data frame
df = pd.DataFrame({"Product Name":product_name,"Brand Name":brand_name,"Price":price,"Return/Exchange":return_exchange, "Expected Delivery":expected_delivery, "Availability":availability, "Product URL":product_url})
df

,Product Name,Brand Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence Frontier guitar with Online Guitar lea...,Visit the Kadence Store,-,7 Days Replacement,"Sunday, Nov 13",,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence Slowhand Premium Jumbo Semi Acoustic G...,Visit the Kadence Store,-,7 Days Replacement,"Wednesday, Nov 16",,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Festra R-40- Trussrod Acoustic Guitar Kit with...,Brand: FESTRA,,7 Days Replacement,-,Only 2 left in stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Vault EA40 41 inch Premium Solid Spruce-Top Cu...,Visit the VAULT Store,,7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the Juârez Store,-,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
120,Tanglewood Roadster II TWR2 DE Semi Acoustic G...,Visit the Tanglewood Store,,7 Days Replacement,-,In stock.,https://www.amazon.in/Tanglewood-Roadster-TWR2...
121,"Blueberry B-D38, 38 Inch Acoustic Guitar Kit I...",Visit the blueberry Store,,7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
122,Cort AD810 Dreadnought Acoustic Guitar with Gi...,Brand: Cort,,7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
123,Right Search ® Mini Musical Guitar Toy _g1,Brand: Right Search,,7 Days Returnable,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


3) Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [17]:
#making a list of search items
search_item = ["fruits", "cars", "Machine Learning","Guitar","Cakes"]

#creating empty list
imgsrc = []
imgurl = []

#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#scraping data for each search item
for item in search_item:
    #opening the page on automated chrome browser
    driver.get('https://images.google.com/')
    time.sleep(1)

    #accessing search bar
    search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
    search_bar.send_keys(item)
    
    #clicking search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']")
    driver.execute_script("arguments[0].click();", search_button)
    time.sleep(1)
    
    #scraping 10 images
    image_tag = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    for i in image_tag[0:10]:
        imgurl.append(i.get_attribute('src'))
    
#creating a data frame
df = pd.DataFrame({"Image":imgurl})
df    
    
    

,Image
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


4) Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [20]:
# getting input from user
item = input(" Enter the name of Smartphone that has to be searched : ")

 Enter the name of Smartphone that has to be searched : one plus


In [22]:
item

'one plus'

In [37]:
page1_urls = []

#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.flipkart.com/')
time.sleep(1)

#close the login prompt
login_dialog_box = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')
login_dialog_box.click()
time.sleep(2)

#enter sneakers in search field
search = driver.find_element(By.XPATH,'//input[@class="_3704LK"]')
search.send_keys(item)
time.sleep(2)

#click search icon
search_icon = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')
search_icon.click()
time.sleep(1)

# Fetching urls of phones coming on 1st page
urls = driver.find_elements(By.XPATH,'//a[@class="s1Q9rs"]')

for url in urls:
    page1_urls.append(url.get_attribute("href"))
len(page1_urls)

40

In [42]:
#initializing blank arrays
# “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
# “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”.
Brand = []
Product = []
Colour = []
RAM = []
ROM = []
PrimCamera = []
SecCamera = []
DispSize = []
Price = []
BatteryCap = []
ProductUrl = []

for i in page1_urls:
    driver.get(i)
    time.sleep(1)
    
    ProductUrl.append(i)
    
    #clicking read more button
    morebtn = driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")
    driver.execute_script("arguments[0].click();", morebtn)
    time.sleep(1)
    
    
    try:
        brandtag = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        Brand.append(brandtag.text)
    except:
        Brand.append('-')
        
    try:
        prodtag = driver.find_element(By.XPATH,"//span[@class='B_NuCI']")
        Product.append(prodtag.text)
    except:
        Product.append('-')
        
    try:
        Colourtag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        Colour.append(Colourtag.text)
    except NoSuchElementException:
        Colour.append('-')
        
    try:
        RAMtag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]/td[2]/ul/li")
        RAM.append(RAMtag.text)
    except:
        RAM.append('-')
        
    try:
        ROMtag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]/td[2]/ul/li")
        ROM.append(ROMtag.text)
    except:
        ROM.append('-')
        
    try:
        pricetag = driver.find_element(By.XPATH,"//div[@class='_30jeq3 _16Jk6d']")
        Price.append(pricetag.text)
    except:
        Price.append('-')
        
    try:
        PrimCameratag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table/tbody/tr[1]/td[2]/ul/li")
        PrimCamera.append(PrimCameratag.text.split(sep, 1)[0])
    except:
        PrimCamera.append('-')
    
    try:
        SecCameratag = driver.find_element(By.XPATH,"//div[@class='_2418kt']/ul/li[3]")
        SecCamera.append(SecCameratag.text.split(sep, 1)[1])
    except:
        SecCamera.append('-')
    
    try:
        DispSizetag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table/tbody/tr[1]/td[2]/ul/li")
        DispSize.append(DispSizetag.text)
    except:
        DispSize.append('-')
        
    try:
        BatteryCaptag = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][7]/table/tbody/tr[1]/td[2]/ul/li")
        BatteryCap.append(BatteryCaptag.text)
    except:
        BatteryCap.append('-')

In [43]:
df = pd.DataFrame({'Brand Name':Brand,'Smartphone Name':Product,'Price(₹)':Price,'Colour':Colour,'RAM':RAM ,'Storage(ROM)':ROM,'Primary Camera':PrimCamera,'Secondary Camera':SecCamera,'Display Size':DispSize,'Battery Capacity':BatteryCap,'Product URL':ProductUrl})
df

,Brand Name,Smartphone Name,Price(₹),Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Product URL
0,-,"OnePlus Nord 2T 5G (Jade Fog, 128 GB) (8 GB RAM)","₹28,999",Jade Fog,8 GB,128 GB,-,-,17.02 cm (6.7 inch),4500 mAh,https://www.flipkart.com/oneplus-nord-2t-5g-ja...
1,-,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...","₹19,999",Blue Tide,6 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,-,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...","₹19,999",Black Dusk,6 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,-,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...","₹19,990",Black Dusk,6 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,-,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...","₹19,980",Blue Tide,6 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,-,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...","₹21,999",Blue Tide,8 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,-,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...","₹19,999",Black Dusk,6 GB,128 GB,-,-,16.74 cm (6.59 inch),5000 mAh,https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,-,"OnePlus Nord 2T 5G (Gray Shadow, 128 GB) (8 G...","₹28,979",Gray Shadow,8 GB,128 GB,-,-,17.02 cm (6.7 inch),4500 mAh,https://www.flipkart.com/oneplus-nord-2t-5g-gr...
8,-,"OnePlus Nord 2T 5G (Jade Fog, 128 GB) (8 GB RAM)","₹28,990",Jade Fog,8 GB,128 GB,-,-,17.02 cm (6.7 inch),4500 mAh,https://www.flipkart.com/oneplus-nord-2t-5g-ja...
9,-,"OnePlus Nord 2T 5G (Gray Shadow, 256 GB) (12 ...","₹33,900",Gray Shadow,12 GB,256 GB,-,-,17.02 cm (6.7 inch),4500 mAh,https://www.flipkart.com/oneplus-nord-2t-5g-gr...


5) Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [44]:
#user input
city = input('Enter City name that has to be searched : ')
city

Enter City name that has to be searched : Ambala


'Ambala'

In [51]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.google.co.in/maps')
time.sleep(1)

#Sending keyword for seach bar and search button
search_bar = driver.find_element(By.XPATH,"//input[@id='searchboxinput']")                       
search_bar.clear()                                                             
time.sleep(1)
search_bar.send_keys(city)   

#click search button
search_btn = driver.find_element(By.XPATH,"//button[@id='searchbox-searchbutton']")              
search_btn.click()                                                             
time.sleep(1)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*),',url_str)
    #print(latitude_longitude)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/search/Ambala/@30.1094233,76.6759107,15z
Latitude = 30.1094233, Longitude = 76.6759107


6) Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [62]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://trak.in/india-startup-funding-investment-2015/')
time.sleep(1)

#initializing empty arrays
Dates=[]
Company=[]
Industry=[]
SubVertical = []
City=[]
InvestorName=[]
InvestorType=[]
Amount=[]

#Data for Jan 21 - March 21 can be found in table ids = "tablepress-54","tablepress-55" and "tablepress-56"

for i in range(54,57): 
    
#scraping Date
    try:
        date_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[2]")
        for i in date_tag:
            Dates.append(i.text)
    except NoSuchElementException:
        Dates.append('-')
    
#scraping Name
for i in range(54,57): 
    try:
        name_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[3]/a")
        for j in name_tag:
            Company.append(j.text)
    except NoSuchElementException:
        Company.append('-')
    
#scraping Industry
for i in range(54,57): 
    try:
        indstry_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[4]")
        for p in indstry_tag:
            Industry.append(p.text)
    except NoSuchElementException:
        Industry.append('-')
    
#scraping Sub Vertical
for i in range(54,57): 
    try:
        subvrt_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[5]")
        for k in subvrt_tag:
            SubVertical.append(k.text)
    except NoSuchElementException:
        SubVertical.append('-')
        
#scraping City 
for i in range(54,57): 
    try:
        city_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[6]")
        for l in city_tag:
            City.append(l.text)
    except NoSuchElementException:
        City.append('-')
        
#scraping Investor Name 
for i in range(54,57): 
    try:
        invname_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[7]")
        for m in invname_tag:
            InvestorName.append(m.text)
    except NoSuchElementException:
        InvestorName.append('-')
        
#scraping Investment Type 
for i in range(54,57): 
    try:
        invtyp_tag  = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[8]")
        for n in invtyp_tag:
            InvestorType.append(n.text)
    except NoSuchElementException:
        InvestorType.append('-')
        
#scraping Amount  
for i in range(54,57): 
    try:
        amt_tag = driver.find_elements(By.XPATH,f"//*[@class='tablepress tablepress-id-{i} dataTable no-footer']/tbody/tr/td[9]")
        for o in amt_tag:
            Amount.append(o.text)   
    except NoSuchElementException:
        Amount.append('-')
        
df = pd.DataFrame({'Date':Dates,'Startup Name':Company,'Industry/Vertical':Industry,'Sub-Vertical':SubVertical,'City/Location':City ,"Investor's Name":InvestorName,'Investment Type':InvestorType,'Amount(in USD)':Amount})      
df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


7) Write a program to scrap all the available details of best gaming laptops from digit.in.

In [71]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.digit.in/')
time.sleep(1)

#searching for best gaming laptop
best_gam_lap = driver.find_element(By.XPATH,"//div[@class='listing_container']/ul/li[9]").click()
time.sleep(1)

#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []

# Scraping the data of laptop names
name_tags = driver.find_elements(By.XPATH,"//div[@class='TopNumbeHeading']/div/a/h3")
for name in name_tags:
    lap_name.append(name.text)
     
#Scraping the data of operating system
os_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[5]//td[3]")
for os in os_tags:
    ope_sys.append(os.text)

#Scraping data of display
disp_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[4]//td[3]")
for disp in disp_tags:
    display.append(disp.text)

#Scraping data of Processor
pro_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[3]//td[3]")
for pro in pro_tags:
    processor.append(pro.text)

#Scraping data of memory
memo_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[6]//td[3]")
for memo in memo_tags:
    memory.append(memo.text)

#Scraping data of dimensions
dim_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[8]//td[3]")
for dim in dim_tags:
    dimensions.append(dim.text)

#Scraping data of Graph processor
gra_tags = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[7]//td[3]")
for gra in gra_tags:
    graph_proc.append(gra.text)


#DATA FRAMING
df=pd.DataFrame({'Laptop Name':lap_name,'Operating system':ope_sys,'Display':display,'Processor':processor,'Memory':memory,'Dimensions':dimensions,'Graphical Processor':graph_proc})
df

,Laptop Name,Operating system,Display,Processor,Memory,Dimensions,Graphical Processor
0,MSI Titan GT77-12UHS,Windows 11 Home,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",12th Gen Intel Core i9-12900HX 16 core processor,64 GB DDR5 RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...
1,Alienware X17 R2,Windows 11 Home,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",12th Gen Intel Core i9-12900H 14 core processo...,32 GB DDR5 RAM & 1 TB SSD,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...
2,Acer Predator Triton 500 SE PT516-52s,Windows 11,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5 RAM & 2 TB SSD,262 x 359 x 20 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...
3,Omen By HP (16-B1371TX),Windows 11 Home,16.1″ (2560 x 1440) screen,12th Gen Intel Core i7-12700H 14 core processo...,8 GB DDR5 RAM & 1 TB SSD,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card
4,Acer Predator Helios 300 AN515-45,Windows 11 Home,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",AMD Ryzen 9-5900HX 8 core processor,16 GB DDR4 RAM & 512 GB SSD,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,16 GB DDR4 RAM & 1 TB SSD,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card
6,Omen By HP (16-C0141AX),Windows 11 Home,16.1″ (2560 x 1440) screen,AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16 GB DDR4 RAM & 1 TB NVMe,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home,"16″ (2560 x 1600) screen, 165 Hz refresh rate",12th Gen Intel Core i7-12700H 14 core processo...,16 GB DDR5 RAM & 1 TB SSD,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",AMD Ryzen R7-5800H 8 core processor with 4.40 ...,16 GB DDR4 RAM & 512 GB SSD,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home,"15.6″ screen, 165 Hz refresh rate",Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,16 GB DDR4 RAM & 1 TB SSD,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card


8) Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [73]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.forbes.com/billionaires/')
time.sleep(1)

#initializing lists
rank=[]
name=[]
net_worth=[]
country=[]
source=[]
age=[]
industry=[]

Rank = driver.find_elements(By.XPATH,"//div[@class='rank']")
for i in Rank:
    rank.append(i.text)
    
nam=driver.find_elements(By.XPATH,"//div[@class='personName']")
for i in nam:
    name.append(i.text)
    
Net_worth=driver.find_elements(By.XPATH,"//div[@class='netWorth']")
for i in Net_worth:
    net_worth.append(i.text)

Country=driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for i in Country:
    country.append(i.text)
    
Source=driver.find_elements(By.XPATH,"//div[@class='source']")
for i in Source:
    source.append(i.text)
    
Age=driver.find_elements(By.XPATH,"//div[@class='age']")
for i in Age:
    age.append(i.text)
    
Industry=driver.find_elements(By.XPATH,"//div[@class='category']")
for i in Industry:
    industry.append(i.text)
    
df = pd.DataFrame({'Rank':rank,'Name':name,'Age':age,'Net Worth':net_worth,'Country':country,'Source':source,'Industry':industry})
df

,Rank,Name,Age,Net Worth,Country,Source,Industry
0,1.,Elon Musk,50,$219 B,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,58,$171 B,United States,Amazon,Technology
2,3.,Bernard Arnault & family,73,$158 B,France,LVMH,Fashion & Retail
3,4.,Bill Gates,66,$129 B,United States,Microsoft,Technology
4,5.,Warren Buffett,91,$118 B,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,72,$10.3 B,Brazil,beer,Food & Beverage
196,197.,Leon Black,70,$10 B,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,40,$10 B,United States,Airbnb,Technology
198,197.,David Geffen,79,$10 B,United States,"movies, record labels",Media & Entertainment


9) Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [94]:
#take url as user input 
url = input('Enter Youtube video URL:')

Enter Youtube video URL:https://www.youtube.com/watch?v=mEWNgapblkk


In [95]:
url

'https://www.youtube.com/watch?v=mEWNgapblkk'

In [97]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get(url)
time.sleep(1)

#using the method of scroll down so that the 500 comments can be automated
i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1

#initializing blank lists
cmt = []
upvt = []
cmttime = []

#scraping details
cmt_tag = driver.find_elements(By.XPATH,"//*[@id='content-text']")
for i in cmt_tag:
    try: 
        cmt.append(i.text)
    except:
        cmt.append('-')

upvt_tag = driver.find_elements(By.XPATH,"//*[@id='vote-count-middle']")
for j in upvt_tag:
    try: 
        upvt.append(j.text)
    except:
        upvt.append('-')
        
cmttime_tag = driver.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a")
for k in cmttime_tag:
    try: 
        cmttime.append(k.text)
    except:
        cmttime.append('-')
        
df = pd.DataFrame({'Comment':cmt,'Commented At':cmttime,'Number of upvotes':upvt})
df

,Comment,Commented At,Number of upvotes
0,Finally a useful mock interview for a data ana...,4 months ago,3
1,One of the best mock interviews for the role o...,6 days ago,2
2,very good video...really insightful,10 months ago,
3,This is amazing video for those who want a car...,7 days ago,2
4,This video helps me a lot in my interview,10 months ago,
...,...,...,...
85,she's smart,2 months ago,
86,data analytiics kisi ka passion kaise ho sakta...,3 months ago,3
87,Lets be honest here! whenever the interviewer ...,2 months ago,
88,Par jisko excellent english nahi aati sif thod...,4 months ago,1


10) Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [109]:
#connect to the driver
driver = webdriver.Chrome(r'F:\Project Data\flip robo\chromedriver.exe')

#opening the page on automated chrome browser
driver.get('https://www.hostelworld.com/')
time.sleep(1)

#locating the location search bar
search_loc = driver.find_element(By.XPATH,'//*[@id="search-input-field"]')
search_loc.send_keys("London")
time.sleep(2)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search-input-field"]"}
  (Session info: chrome=107.0.5304.89)
Stacktrace:
Backtrace:
	Ordinal0 [0x0087ACD3+2075859]
	Ordinal0 [0x0080EE61+1633889]
	Ordinal0 [0x0070B7BD+571325]
	Ordinal0 [0x0073AC2F+764975]
	Ordinal0 [0x0073AE1B+765467]
	Ordinal0 [0x0076D0F2+970994]
	Ordinal0 [0x00757364+881508]
	Ordinal0 [0x0076B56A+963946]
	Ordinal0 [0x00757136+880950]
	Ordinal0 [0x0072FEFD+720637]
	Ordinal0 [0x00730F3F+724799]
	GetHandleVerifier [0x00B2EED2+2769538]
	GetHandleVerifier [0x00B20D95+2711877]
	GetHandleVerifier [0x0090A03A+521194]
	GetHandleVerifier [0x00908DA0+516432]
	Ordinal0 [0x0081682C+1665068]
	Ordinal0 [0x0081B128+1683752]
	Ordinal0 [0x0081B215+1683989]
	Ordinal0 [0x00826484+1729668]
	BaseThreadInitThunk [0x76D57BA9+25]
	RtlInitializeExceptionChain [0x779FBB3B+107]
	RtlClearBits [0x779FBABF+191]


In [ ]:
#was not able to locate the search bar